In [1]:
 # main

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent.resolve()
sys.path.insert(0, str(PROJECT_ROOT))

from main import main

In [2]:
main()

📄 Gefundene CSV-Dateien: 1
📄 Gefundene PDF-Dateien: 1
🔄 Lade aus bestehender Collection.
📂 Chroma-Verzeichnis gefunden. Vorhandene Daten werden geladen.
📨 Bearbeite Stellenanzeige: Ausschreibung_EnBw
🔄 Verwende gecachte Reduktion.
🎯 Passgenauigkeit laut ChatGPT: 58%
✅ Referenzen gespeichert unter: output/Ausschreibung_EnBw_references.txt
✅ HTML-Kurzprofil gespeichert unter: output/Kurzprofil_Marcel_Russ_Ausschreibung_EnBw.html
